In [1]:
import os

In [2]:
%pwd

'e:\\ML_Projects\\Wine_Quality_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\ML_Projects\\Wine_Quality_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str
    
# from dataclasses import dataclass
# from pathlib import Path
# from typing import Optional

# @dataclass(frozen=True)
# class ModelTrainerConfig:
#     root_dir: Path
#     train_data_path: Path
#     test_data_path: Path
#     model_name: str
#     target_column: str

#     # Model hyperparameters with default values
#     linear_regression: dict = None  # No hyperparameters for LinearRegression

#     ridge: dict = None or {"alphaaaa": 1.0, "solver": "auto"}

#     lasso: dict = None or {"alphaaaa": 0.1}

#     elastic_net: dict = None or {"alphaaaa": 0.5, "l1_ratio": 0.7}

#     decision_tree: dict = None or {
#         "criterion": "squared_error",
#         "max_depth": 5,
#         "min_samples_split": 2
#     }

#     random_forest: dict = None or {
#         "n_estimators": 100,
#         "max_depth": 10,
#         "random_state": 42
#     }

#     gradient_boosting: dict = None or {
#         "n_estimators": 100,
#         "learning_rate": 0.1,
#         "max_depth": 3
#     }

#     svr: dict = None or {
#         "kernel": "rbf",
#         "C": 1.0,
#         "epsilon": 0.1
#     }

#     knn: dict = None or {
#         "n_neighbors": 5,
#         "weights": "uniform"
#     }

#     xgb: dict = None or {
#         "n_estimators": 100,
#         "learning_rate": 0.1,
#         "max_depth": 3,
#         "objective": "reg:squarederror"
#     }

#     lightgbm: dict = None or {
#         "n_estimators": 100,
#         "learning_rate": 0.1,
#         "max_depth": -1,
#         "objective": "regression"
#     }


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

    # def get_model_trainer_config(self) -> ModelTrainerConfig:
    #     config = self.config.model_trainer
    #     schema = self.schema.TARGET_COLUMN

    #     model_name = config.model_name

    #     if model_name == "LinearRegression":
    #         params = {}
    #     elif model_name == "Ridge":
    #         params = self.params.Ridge
    #     elif model_name == "Lasso":
    #         params = self.params.Lasso
    #     elif model_name == "ElasticNet":
    #         params = self.params.ElasticNet
    #     elif model_name == "DecisionTreeRegressor":
    #         params = self.params.DecisionTreeRegressor
    #     elif model_name == "RandomForestRegressor":
    #         params = self.params.RandomForestRegressor
    #     elif model_name == "GradientBoostingRegressor":
    #         params = self.params.GradientBoostingRegressor
    #     elif model_name == "SVR":
    #         params = self.params.SVR
    #     elif model_name == "KNeighborsRegressor":
    #         params = self.params.KNeighborsRegressor
    #     elif model_name == "XGBRegressor":
    #         params = self.params.XGBRegressor
    #     elif model_name == "LightGBMRegressor":
    #         params = self.params.LightGBMRegressor
    #     else:
    #         raise ValueError(f"Unsupported model name: {model_name}")

    #     model_trainer_config = ModelTrainerConfig(
    #         root_dir=config.root_dir,
    #         train_data_path=config.train_data_path,
    #         test_data_path=config.test_data_path,
    #         model_name=model_name,
    #         alphaaaa=getattr(params, "alphaaaa", None),
    #         l1_ratio=getattr(params, "l1_ratio", None),
    #         criterion=getattr(params, "criterion", None),
    #         max_depth=getattr(params, "max_depth", None),
    #         min_samples_split=getattr(params, "min_samples_split", None),
    #         n_estimators=getattr(params, "n_estimators", None),
    #         learning_rate=getattr(params, "learning_rate", None),
    #         random_state=getattr(params, "random_state", None),
    #         kernel=getattr(params, "kernel", None),
    #         C=getattr(params, "C", None),
    #         epsilon=getattr(params, "epsilon", None),
    #         n_neighbors=getattr(params, "n_neighbors", None),
    #         weights=getattr(params, "weights", None),
    #         objective=getattr(params, "objective", None),
    #         target_column=schema.name,
    #     )

    #     return model_trainer_config
    

In [7]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        
# import pandas as pd
# import os
# import joblib
# from mlProject import logger
# from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
# from sklearn.svm import SVR
# from sklearn.neighbors import KNeighborsRegressor

# class ModelTrainer:
#     def __init__(self, config: ModelTrainerConfig):
#         self.config = config

#     def _get_model_instance(self):
#         name = self.config.model_name
#         if name == "LinearRegression":
#             return LinearRegression()
#         elif name == "Ridge":
#             return Ridge(alphaaaa=self.config.alphaaaa)
#         elif name == "Lasso":
#             return Lasso(alphaaaa=self.config.alphaaaa)
#         elif name == "ElasticNet":
#             return ElasticNet(alphaaaa=self.config.alphaaaa, l1_ratio=self.config.l1_ratio)
#         elif name == "DecisionTreeRegressor":
#             return DecisionTreeRegressor(criterion=self.config.criterion,
#                                          max_depth=self.config.max_depth,
#                                          min_samples_split=self.config.min_samples_split)
#         elif name == "RandomForestRegressor":
#             return RandomForestRegressor(n_estimators=self.config.n_estimators,
#                                          max_depth=self.config.max_depth,
#                                          random_state=self.config.random_state)
#         elif name == "GradientBoostingRegressor":
#             return GradientBoostingRegressor(n_estimators=self.config.n_estimators,
#                                              learning_rate=self.config.learning_rate,
#                                              max_depth=self.config.max_depth)
#         elif name == "SVR":
#             return SVR(kernel=self.config.kernel,
#                        C=self.config.C,
#                        epsilon=self.config.epsilon)
#         elif name == "KNeighborsRegressor":
#             return KNeighborsRegressor(n_neighbors=self.config.n_neighbors,
#                                        weights=self.config.weights)
#         else:
#             raise ValueError(f"Unsupported model: {name}")

#     def train(self):
#         # Load Data
#         train_data = pd.read_csv(self.config.train_data_path)
#         test_data = pd.read_csv(self.config.test_data_path)

#         train_x = train_data.drop(columns=[self.config.target_column])
#         test_x = test_data.drop(columns=[self.config.target_column])
#         train_y = train_data[self.config.target_column]
#         test_y = test_data[self.config.target_column]

#         # Get model
#         model = self._get_model_instance()

#         # Train model
#         model.fit(train_x, train_y)

#         # Save model
#         model_path = os.path.join(self.config.root_dir, self.config.model_name + ".pkl")
#         joblib.dump(model, model_path)

#         logger.info(f"Model {self.config.model_name} trained and saved at {model_path}")

        
        



In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-05-27 06:46:42,924: INFO: common: YAML file: E:\ML_Projects\Wine_Quality_Project\config\config.yaml loaded successfully]
[2025-05-27 06:46:42,940: INFO: common: YAML file: params.yaml loaded successfully]
[2025-05-27 06:46:42,971: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-05-27 06:46:42,978: INFO: common: Created directory at: artifacts]
[2025-05-27 06:46:42,990: INFO: common: Created directory at: artifacts/model_trainer]
